In [68]:
import math

def cubic_iteration_general(a, b, c, d, x0, max_iter=50, tol=1e-8, pick="positive"):
    """
    Surrogate iterative solver for cubic a*x^3 + b*x^2 + c*x + d = 0
    Uses lagged x[k-1]^3 scheme.

    Args:
        a, b, c, d : coefficients of cubic
        x0 : initial guess
        max_iter : max iterations
        tol : tolerance for convergence
        pick : "positive" (default) -> choose root with +sqrt
               "negative" -> choose root with -sqrt

    Returns:
        (root, history) : approx root and iteration history
    """
    history = [x0]
    x = x0

    for k in range(max_iter):
        # Quadratic coefficients in x_k
        A = b
        B = c
        C = a * (x**3) + d

        if A == 0:  # fallback to linear
            if B == 0:
                raise ValueError("Degenerate equation: no x_k dependence.")
            x_new = -C / B
        else:
            disc = B*B - 4*A*C
            if disc < 0:
                sqrt_disc = 0.0  # clamp to stay real
            else:
                sqrt_disc = math.sqrt(disc)

            if pick == "positive":
                x_new = (-B + sqrt_disc) / (2*A)
            else:
                x_new = (-B - sqrt_disc) / (2*A)

        history.append(x_new)
        if abs(x_new - x) < tol:
            break
        x = x_new

    return x, history


import numpy as np

cubics = [
    ((1, -6, 11, -6), "(x-1)(x-2)(x-3)"),      # roots: 1,2,3
    ((1, -6, 5, 12), "(x+1)(x-3)(x-4)"),       # roots: -1,3,4
    ((1, -10, 31, -30), "(x-2)(x-3)(x-5)"),    # roots: 2,3,5
    ((1, -3, -6, 8), "(x+2)(x-1)(x-4)"),       # roots: -2,1,4
    ((1, -4, -11, 30), "(x+3)(x-2)(x-5)"),     # roots: -3,2,5
    ((1, -4, -7, 10), "(x-1)(x+2)(x-5)"),      # roots: 1,-2,5
    ((1, 6, 11, 6), "(x+1)(x+2)(x+3)"),        # roots: -1,-2,-3
    ((1, -15, 74, -120), "(x-4)(x-5)(x-6)"),   # roots: 4,5,6
    ((1, -1, -32, 40), "(x-2)(x-4)(x+5)"),     # roots: 2,4,-5
    ((1, -12, 35, 42), "(x+1)(x-6)(x-7)"),     # roots: -1,6,7
]

# Starting guesses for variety
x0_guesses = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

for (coeffs, factored), x0 in zip(cubics, x0_guesses):
    a, b, c, d = coeffs
    root, hist = cubic_iteration_general(a, b, c, d, x0=x0, max_iter=100)

    # True roots via numpy
    true_roots = np.roots([a, b, c, d])
    true_roots_real = [r.real for r in true_roots if abs(r.imag) < 1e-8]

    print(f"\nCubic: {factored} -> {a}x^3 + {b}x^2 + {c}x + {d} = 0")
    print(f"Starting guess: {x0}")
    print(f"Approx root: {root}")
    print(f"Iterations: {hist}")
    print(f"True real roots: {true_roots_real}")



Cubic: (x-1)(x-2)(x-3) -> 1x^3 + -6x^2 + 11x + -6 = 0
Starting guess: 0.1
Approx root: 0.9166666666666666
Iterations: [0.1, 0.9166666666666666, 0.9166666666666666]
True real roots: [np.float64(3.0000000000000018), np.float64(1.999999999999998), np.float64(1.0000000000000002)]

Cubic: (x+1)(x-3)(x-4) -> 1x^3 + -6x^2 + 5x + 12 = 0
Starting guess: 0.1
Approx root: -1.0000000017790027
Iterations: [0.1, -1.0577070233379324, -0.9891760891702944, -1.0018882433336624, -0.999666111646429, -1.000058900578704, -0.9999896051298546, -1.0000018343685932, -0.9999996762872646, -1.0000000571257572, -0.9999999899189834, -1.0000000017790027, -0.9999999996860586]
True real roots: [np.float64(4.000000000000006), np.float64(2.999999999999997), np.float64(-1.0000000000000004)]

Cubic: (x-2)(x-3)(x-5) -> 1x^3 + -10x^2 + 31x + -30 = 0
Starting guess: 0.1
Approx root: 1.55
Iterations: [0.1, 1.55, 1.55]
True real roots: [np.float64(5.0), np.float64(3.000000000000009), np.float64(1.999999999999996)]

Cubic: (x+2

In [59]:
import math

def cubic_iteration_general_relaxed(a, b, c, d, x0, max_iter=50, tol=1e-8,
                                    pick="positive", lam=0.5):
    """
    Relaxed surrogate iteration for cubic a*x^3 + b*x^2 + c*x + d = 0.
    Uses lagged x[k-1]^3 surrogate + relaxation.

    Args:
        a, b, c, d : cubic coefficients
        x0 : initial guess
        max_iter : maximum iterations
        tol : convergence tolerance
        pick : "positive" (default) or "negative" root of quadratic
        lam : relaxation factor (0<lam<=1)

    Returns:
        (root, history)
    """
    history = [x0]
    x = x0

    for k in range(max_iter):
        # Quadratic coefficients in x_k
        A = b
        B = c
        C = a * (x**3) + d

        if A == 0:  # linear fallback
            if B == 0:
                raise ValueError("Degenerate: no x_k dependence.")
            g = -C / B
        else:
            disc = B*B - 4*A*C
            sqrt_disc = math.sqrt(disc) if disc >= 0 else 0.0

            if pick == "positive":
                g = (-B + sqrt_disc) / (2*A)
            else:
                g = (-B - sqrt_disc) / (2*A)

        # Relaxed update
        x_new = (1 - lam) * x + lam * g
        history.append(x_new)

        if abs(x_new - x) < tol:
            break
        x = x_new

    return x, history

# Test on f(x) = x^3 - 6x^2 + 11x - 6 = (x-1)(x-2)(x-3)
root, hist = cubic_iteration_general_relaxed(1, -6, 11, -6, x0=20, lam=0.2, max_iter=100)
print("Approx root:", root)

# Test on f(x) = x^3 - 6x^2 + 5x + 12 = (x-1)(x-3)(x-4)
root2, hist2 = cubic_iteration_general_relaxed(1, -6, 5, 12, x0=5, lam=0.3, max_iter=100)
print("Approx root:", root2)

Approx root: 0.9166667164854967
Approx root: -0.9999999791915551
